In [7]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import PIL.Image as pilimg
import numpy as np

In [4]:
im = pilimg.open("./images/cat1.jpg")

In [5]:
im.show()

In [8]:
pix = np.array(im)

In [9]:
print(pix[:10])

[[[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [237 241 244]
  [237 241 244]
  [237 241 244]]

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [237 241 244]
  [237 241 244]
  [237 241 244]]

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [237 241 244]
  [237 241 244]
  [237 241 244]]

 ...

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [237 241 244]
  [237 241 244]
  [237 241 244]]

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [237 241 244]
  [237 241 244]
  [237 241 244]]

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [237 241 244]
  [237 241 244]
  [237 241 244]]]


In [10]:
print(pix.shape)

(630, 1200, 3)


In [4]:
import tensorflow as tf 
import tensorflow.keras as keras 
from tensorflow.keras.datasets import mnist 
import PIL.Image as pilimg

print(type(mnist) ) 

(train_images, train_labels), (test_images, test_labels)=mnist.load_data() 
print( train_images.shape ) # 28 by 28 이미지가 60000개 있음 

#앞에 천개만 image1.jpg image2.jpg 형태로 저장하기 
for i in range(1000):
    pilimg.fromarray(train_images[i]).save("./images/numbers/image{}.png".format(i))    


<class 'module'>
(60000, 28, 28)


In [8]:
import numpy as np
import os
import random
from PIL import Image as pilimg
import imghdr
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:

# ======================================================================
# 1. 데이터 로딩 및 전처리 (Keras 코드와 동일)
# ======================================================================

# 데이터 만드는 수 
def makeData(folder, label, isTrain):
    if isTrain == 'train':
        path = "./data/flowers_/train/" + folder
    else:
        path = "./data/flowers_/test/" + folder
    data = [] 
    labels = [] 
    i=1 
    for filename in os.listdir(path):
        if i%10==0:
            print(i)
        i += 1
        try: 
            kind = imghdr.what(path + "/" + filename)
            if kind in ["gif", "png", "jpeg", "jpg"]:
                img = pilimg.open(path + "/" + filename).convert('RGB') # PyTorch는 3채널 RGB를 선호
                resize_img = img.resize((150, 150))
                pixel = np.array(resize_img)
                if pixel.shape==(150, 150, 3):
                    data.append(pixel)
                    labels.append(label)
        except Exception as e:
            print(f"{filename} error: {e}")

    np.savez("imagedata{}.npz".format(str(label) + '_' + isTrain), data=data, targets=labels)
    print("파일저장완료")

    # makeData('daisy'    ,0,'train')
    # makeData('dandelion',1,'train')
    # makeData('rose'     ,2,'train')
    # makeData('sunflower',3,'train')
    # makeData('tulip'    ,4,'train')    

    # makeData('daisy'    ,0,'train')
    # makeData('dandelion',1,'train')
    # makeData('rose'     ,2,'train')
    # makeData('sunflower',3,'train')
    # makeData('tulip'    ,4,'train')    



In [ ]:
# # 이미 생성된 npz 파일이 있다고 가정하고 로딩하는 코드
# # makeData 함수는 위 Keras 코드의 실행 결과와 동일한 데이터를 생성

# def dataCreate():
#     flowers = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
#     label=0 
#     for flower in flowers:
#         makeData(flower, label, 'train')
#         makeData(flower, label, 'test')
#         label += 1

# def load_and_concat_data(mode):
#     datas = []
#     targets = []
#     for label in range(5):
#         npz_file = np.load(f"imagedata{label}_{mode}.npz")
#         datas.append(npz_file["data"])
#         targets.append(npz_file["targets"])
    
#     return np.concatenate(datas, axis=0), np.concatenate(targets, axis=0)

# dataCreate()

# trainData_np, trainTarget_np = load_and_concat_data('train')
# testData_np, testTarget_np = load_and_concat_data('test')

파일저장완료
파일저장완료
파일저장완료
파일저장완료
파일저장완료
파일저장완료
파일저장완료
파일저장완료
파일저장완료
파일저장완료


In [18]:

# 데이터 형태 변환 (PyTorch에 맞게)
# PyTorch는 `channels_first` (C, H, W) 형태를 선호합니다.
# 데이터 정규화: 0-255 -> 0.0-1.0
trainData_tensor = torch.from_numpy(trainData_np).float().permute(0, 3, 1, 2) / 255.0
testData_tensor = torch.from_numpy(testData_np).float().permute(0, 3, 1, 2) / 255.0

# PyTorch에서는 One-Hot Encoding을 사용하지 않고, CrossEntropyLoss에 정수 라벨을 바로 전달
trainTarget_tensor = torch.from_numpy(trainTarget_np).long()
testTarget_tensor = torch.from_numpy(testTarget_np).long()

print(f"Train Data Shape: {trainData_tensor.shape}")
print(f"Test Data Shape: {testData_tensor.shape}")

# 데이터 로더 생성
train_dataset = TensorDataset(trainData_tensor, trainTarget_tensor)
test_dataset = TensorDataset(testData_tensor, testTarget_tensor)

train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)


Train Data Shape: torch.Size([3027, 3, 150, 150])
Test Data Shape: torch.Size([1296, 3, 150, 150])


In [21]:




# ======================================================================
# 2. 모델 정의 (Fully-Connected Network)
# ======================================================================

# Keras의 Sequential 모델과 동일한 구조의 PyTorch 모델 정의
# 입력 형태 (150*150*3) -> Dense(256) -> Dense(64) -> Dense(32) -> Dense(5)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # (?, 3, 150,150) 
        #1단계
        self.conv1 = nn.Conv2d( 3, 32, kernel_size=3,  stride=1, padding=1)
        self.pool1 = nn.MaxPool2d( kernel_size=2, stride=2)
        #  150/2 = 75 by 75 

        #2단계
        self.conv2 = nn.Conv2d( 32, 64, kernel_size=3,  stride=1, padding=1)
        self.pool2 = nn.MaxPool2d( kernel_size=2, stride=2)
        #  75/2= 37.5 = > 자름  37 by 37
        # 64 *  37 *  37
        self.fc_input_size =64 *  37 *  37
        

        self.fc1 = nn.Linear(self.fc_input_size, 256) #150*150*3 -> 256 입력레이어
        self.relu = nn.ReLU()   
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 5) # 5개의 클래스 (daisy, dandelion, ...)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        #중간에 평탄화 작업 
        x = x.view(-1, self.fc_input_size)

        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


# 모델 인스턴스 생성
model = SimpleCNN()
print(model)

# ======================================================================
# 3. 모델 컴파일 및 학습
# ======================================================================

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss() # Keras의 'categorical_crossentropy'와 동일
optimizer = optim.SGD(model.parameters(), lr=0.01) # learning rate는 기본값으로 설정

epochs = 3
# 학습 루프
for epoch in range(epochs):
    model.train() # 모델을 학습 모드로 설정
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 순전파 (forward pass)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 역전파 (backward pass) 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}')

# ======================================================================
# 4. 모델 평가
# ======================================================================

# 모델을 평가 모드로 설정
model.eval()
train_correct = 0
train_total = 0
test_correct = 0
test_total = 0

# 훈련셋 평가
with torch.no_grad(): # 평가 시에는 그래디언트를 계산하지 않음
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

# 테스트셋 평가
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

train_acc = 100 * train_correct / train_total
test_acc = 100 * test_correct / test_total

print(f'\n훈련셋 정확도: {train_acc:.2f}%')
print(f'테스트셋 정확도: {test_acc:.2f}%')

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=87616, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=5, bias=True)
)
Epoch 1/3, Loss: 1.6122
Epoch 2/3, Loss: 1.6110
Epoch 3/3, Loss: 1.6095

훈련셋 정확도: 24.38%
테스트셋 정확도: 27.01%


In [24]:
import os
import shutil
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pickle

#pip install tqdm
from tqdm import tqdm # 학습 진행 상황 시각화를 위해 추가

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
# 이후 PyTorch, NumPy, TensorFlow 등을 import 합니다.
# 경고 무시 설정
warnings.filterwarnings('ignore')

In [25]:


# 1. 경로 설정 및 하이퍼파라미터
original_dataset_dir = './data/cats_and_dogs/train'
base_dir = './data/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

model_save_path_pth = 'cats_and_dogs_model.pth'
history_filepath = 'cats_and_dogs_history.pkl'


In [26]:
batch_size = 16
img_height = 180
img_width = 180
num_epochs = 30 # 예시로 epoch 수를 30으로 설정했습니다.
learning_rate = 0.001 # learning_rate 추가

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 이미지 복사 함수 
def ImageCopyMove():
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir, ignore_errors=True)
    os.makedirs(train_dir)
    os.makedirs(validation_dir)
    os.makedirs(test_dir)
    
    train_cats_dir = os.path.join(train_dir, 'cats')
    train_dogs_dir = os.path.join(train_dir, 'dogs')
    validation_cats_dir = os.path.join(validation_dir, 'cats')
    validation_dogs_dir = os.path.join(validation_dir, 'dogs')
    test_cats_dir = os.path.join(test_dir, 'cats')
    test_dogs_dir = os.path.join(test_dir, 'dogs')

    os.makedirs(train_cats_dir)
    os.makedirs(train_dogs_dir)
    os.makedirs(validation_cats_dir)
    os.makedirs(validation_dogs_dir)
    os.makedirs(test_cats_dir)
    os.makedirs(test_dogs_dir)
    
    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_cats_dir, fname)
        shutil.copyfile(src, dst)
        
    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(validation_cats_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(test_cats_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_dogs_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(validation_dogs_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(test_dogs_dir, fname)
        shutil.copyfile(src, dst)

    print("이미지 복사 및 폴더 생성 완료!")


사용 중인 디바이스: cpu


In [27]:
ImageCopyMove

<function __main__.ImageCopyMove()>